In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + input)), negative_slope=0.1)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
   ConvTranspose2d-5            [-1, 256, 8, 8]       1,179,904
       BatchNorm2d-6            [-1, 256, 8, 8]             512
   ConvTranspose2d-7          [-1, 128, 16, 16]         295,040
       BatchNorm2d-8          [-1, 128, 16, 16]             256
   ConvTranspose2d-9            [-1, 2, 32, 32]           2,306
      BatchNorm2d-10            [-1, 2, 32, 32]               4
Total params: 2,662,278
Trainable params: 2,662,278
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 6.78
Params size (MB): 10.16
Estima

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00288764 (0.02315031), PSNR 25.39456367 (20.42781895), SSIM 0.76680338 (0.57156759)
Finished training epoch 0
Validate: MSE 0.00325140 (0.00277209), PSNR 24.87929726 (25.64583412), SSIM 0.70451236 (0.77522349)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00254702 (0.00276511), PSNR 25.93968201 (25.60643250), SSIM 0.76885778 (0.76922117)
Finished training epoch 1
Validate: MSE 0.00325793 (0.00276975), PSNR 24.87058258 (25.64909911), SSIM 0.70450664 (0.77525717)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00312165 (0.00277448), PSNR 25.05615997 (25.59478971), SSIM 0.76071715 (0.76913087)
Finished training epoch 2
Validate: MSE 0.00325863 (0.00276654), PSNR 24.86965370 (25.65384613), SSIM 0.70425177 (0.77515539)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00214311 (0.00276698), PSNR 26.68955803 (25.60482202), SSIM 0.80303925 (0.76929482)
Finished training epoch 3
Validate: MSE 0.00340562 (0.0

Epoch: 30, MSE 0.00212266 (0.00235434), PSNR 26.73120117 (26.30236923), SSIM 0.77615970 (0.77626344)
Finished training epoch 30
Validate: MSE 0.00271908 (0.00235179), PSNR 25.65578079 (26.34756111), SSIM 0.71918130 (0.78092380)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00221343 (0.00233790), PSNR 26.54934692 (26.33516418), SSIM 0.78721964 (0.77647060)
Finished training epoch 31
Validate: MSE 0.00276024 (0.00239869), PSNR 25.59053040 (26.26254875), SSIM 0.72093058 (0.78209695)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00250771 (0.00234242), PSNR 26.00722885 (26.32620222), SSIM 0.78503764 (0.77674256)
Finished training epoch 32
Validate: MSE 0.00295985 (0.00254316), PSNR 25.28730011 (26.01930189), SSIM 0.71747041 (0.78001269)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00255377 (0.00233030), PSNR 25.92818451 (26.34688287), SSIM 0.77662551 (0.77694579)
Finished training epoch 33
Validate: MSE 0.00275808 (0.00239235), PSNR 

Epoch: 60, MSE 0.00233798 (0.00228622), PSNR 26.31158829 (26.43520863), SSIM 0.76132476 (0.77752936)
Finished training epoch 60
Validate: MSE 0.00246225 (0.00239839), PSNR 26.08666992 (26.26503179), SSIM 0.72396713 (0.77911728)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00248162 (0.00226781), PSNR 26.05263901 (26.46893051), SSIM 0.76706487 (0.77779186)
Finished training epoch 61
Validate: MSE 0.00255426 (0.00239575), PSNR 25.92734528 (26.27425800), SSIM 0.72474587 (0.78044775)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00279991 (0.00226705), PSNR 25.52855682 (26.46795454), SSIM 0.75642759 (0.77784616)
Finished training epoch 62
Validate: MSE 0.00248159 (0.00226348), PSNR 26.05269623 (26.50900204), SSIM 0.72373557 (0.78201355)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00196026 (0.00226370), PSNR 27.07685471 (26.47627039), SSIM 0.78918791 (0.77764608)
Finished training epoch 63
Validate: MSE 0.00254113 (0.00228630), PSNR 

Epoch: 90, MSE 0.00201682 (0.00223520), PSNR 26.95333290 (26.53265586), SSIM 0.77086389 (0.77777260)
Finished training epoch 90
Validate: MSE 0.00255179 (0.00223095), PSNR 25.93154716 (26.57586032), SSIM 0.72628665 (0.78421846)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00215200 (0.00224796), PSNR 26.67157936 (26.50284244), SSIM 0.78644025 (0.77775119)
Finished training epoch 91
Validate: MSE 0.00265020 (0.00233122), PSNR 25.76721191 (26.38818755), SSIM 0.72023278 (0.78129041)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00252136 (0.00223437), PSNR 25.98365784 (26.53221726), SSIM 0.75465059 (0.77783710)
Finished training epoch 92
Validate: MSE 0.00255879 (0.00232365), PSNR 25.91966057 (26.39460263), SSIM 0.72017431 (0.77965481)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00190310 (0.00223365), PSNR 27.20537376 (26.52883509), SSIM 0.79171813 (0.77765469)
Finished training epoch 93
Validate: MSE 0.00268085 (0.00223226), PSNR 

Epoch: 120, MSE 0.00192435 (0.00221572), PSNR 27.15715981 (26.56757332), SSIM 0.78345948 (0.77790547)
Finished training epoch 120
Validate: MSE 0.00254210 (0.00222989), PSNR 25.94807243 (26.57493398), SSIM 0.72070092 (0.78166908)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00263762 (0.00222134), PSNR 25.78787804 (26.55684011), SSIM 0.78541958 (0.77743354)
Finished training epoch 121
Validate: MSE 0.00261480 (0.00223143), PSNR 25.82561493 (26.57731239), SSIM 0.72217119 (0.78378432)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00191971 (0.00222257), PSNR 27.16763306 (26.55693652), SSIM 0.79908788 (0.77732660)
Finished training epoch 122
Validate: MSE 0.00241885 (0.00225979), PSNR 26.16391182 (26.51260027), SSIM 0.71855527 (0.77498665)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00235601 (0.00221801), PSNR 26.27823067 (26.56097600), SSIM 0.78557312 (0.77755418)
Finished training epoch 123
Validate: MSE 0.00249093 (0.00225

Validate: MSE 0.00265503 (0.00228848), PSNR 25.75930214 (26.46793644), SSIM 0.72041905 (0.78189919)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00217515 (0.00220189), PSNR 26.62510872 (26.59427129), SSIM 0.77365851 (0.77717020)
Finished training epoch 150
Validate: MSE 0.00258524 (0.00234477), PSNR 25.87498856 (26.34604700), SSIM 0.71744424 (0.77384416)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00214762 (0.00220376), PSNR 26.68042564 (26.59108284), SSIM 0.78074849 (0.77714152)
Finished training epoch 151
Validate: MSE 0.00244497 (0.00221896), PSNR 26.11726379 (26.59389651), SSIM 0.72650301 (0.78147233)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00229640 (0.00220013), PSNR 26.38952827 (26.59792266), SSIM 0.76258600 (0.77745855)
Finished training epoch 152
Validate: MSE 0.00254740 (0.00221533), PSNR 25.93902779 (26.60272713), SSIM 0.72440618 (0.78170440)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

Epoch: 179, MSE 0.00213444 (0.00217745), PSNR 26.70715141 (26.64385142), SSIM 0.78458565 (0.77728064)
Finished training epoch 179
Validate: MSE 0.00254436 (0.00228216), PSNR 25.94421959 (26.47750013), SSIM 0.71662796 (0.78158700)
Finished validation.
Starting training epoch 180
Epoch: 180, MSE 0.00198270 (0.00218799), PSNR 27.02743912 (26.62262189), SSIM 0.77921551 (0.77725882)
Finished training epoch 180
Validate: MSE 0.00274918 (0.00222670), PSNR 25.60795975 (26.58832584), SSIM 0.72246259 (0.78342438)
Finished validation.
Starting training epoch 181
Epoch: 181, MSE 0.00193648 (0.00218838), PSNR 27.12985802 (26.62254710), SSIM 0.77536649 (0.77703935)
Finished training epoch 181
Validate: MSE 0.00270799 (0.00227321), PSNR 25.67353439 (26.50223893), SSIM 0.71960843 (0.78280587)
Finished validation.
Starting training epoch 182
Epoch: 182, MSE 0.00234330 (0.00219165), PSNR 26.30172729 (26.61565130), SSIM 0.77909684 (0.77697890)
Finished training epoch 182
Validate: MSE 0.00252109 (0.00244

Validate: MSE 0.00259231 (0.00221713), PSNR 25.86312103 (26.60373585), SSIM 0.71815991 (0.78096048)
Finished validation.
Starting training epoch 209
Epoch: 209, MSE 0.00212187 (0.00217953), PSNR 26.73282051 (26.63751933), SSIM 0.77168685 (0.77689471)
Finished training epoch 209
Validate: MSE 0.00240457 (0.00222219), PSNR 26.18962860 (26.58701730), SSIM 0.72250080 (0.77812572)
Finished validation.
Starting training epoch 210
Epoch: 210, MSE 0.00186540 (0.00217637), PSNR 27.29228973 (26.64380863), SSIM 0.76566124 (0.77669933)
Finished training epoch 210
Validate: MSE 0.00265969 (0.00227313), PSNR 25.75169563 (26.50063833), SSIM 0.71888369 (0.78142229)
Finished validation.
Starting training epoch 211
Epoch: 211, MSE 0.00254948 (0.00217770), PSNR 25.93548775 (26.64246930), SSIM 0.76214296 (0.77666010)
Finished training epoch 211
Validate: MSE 0.00251778 (0.00220686), PSNR 25.98981476 (26.62302025), SSIM 0.72170877 (0.78136010)
Finished validation.
Starting training epoch 212
Epoch: 212, MS

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00265575 (0.00231692), PSNR 25.75811958 (26.41291968), SSIM 0.72052801 (0.77817538)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()